In [2]:
# !pip install statsmodels
# !pip install lifelines==0.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 228.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 327.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 kB 90.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 229.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 208.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 270.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4032 sha256=641bb3e890046237f3510958cddd8272bde13ff588f7831e3c935080d625ccd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-yy0q4442/wheels/a8/03/64/8557323821d25118c3a2dc1646996f7a962a8970d4b7d22473
Successfully built autograd-gamma


# This notebook uses the dataframes created in notebook 04 and creates the data for table 1

In [3]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from lifelines import CoxPHFitter

import warnings
warnings.filterwarnings("ignore")

In [3]:
#Select NDDs
ndd_list = ['AD', 'PD', 'DEM']

codes = ['moxifloxacin', 'fexofenadine', 'lormetazepam', 'nizatidine', 'lacidipine', 'rifaximin', 'levomepromazine', 'losartan', 'metformin', 'colchicine', 'valproicacid', 'acetylcysteine', 'phenoxymethylpenicillin', 'erdosteine', 'montelukast', 'sildenafil', 'amiodarone', 'clomethiazole', 'gemfibrozil', 'alogliptin', 'mirtazapine', 'aclidiniumbromide', 'clomipramine', 'nebivolol', 'minoxidil', 'clonidine', 'alfuzosin', 'rizatriptan', 'mecysteine', 'rimonabant', 'naratriptan', 'paracetamolandmetoclopramide', 'orlistat', 'griseofulvin', 'telmisartan', 'pravastatin', 'zaleplon', 'piroxicam', 'apixaban', 'amphotericin', 'linagliptin', 'amlodipine', 'diltiazem', 'nortriptyline', 'potassiumchlorideandpotassiumbicarbonate', 'propantheline', 'hydrotalcite', 'alprazolam', 'indomethacin', 'ciclesonide', 'nabumetone', 'perindopril', 'digoxin', 'oxerutins', 'amisulpride', 'bisacodyl', 'carbocisteine', 'gabapentin', 'misoprostol', 'ticagrelor', 'apomorphine', 'tetrabenazine', 'tetracycline', 'nalidixicacid', 'methylcellulose', 'flucloxacillin', 'epogam', 'mefenamic', 'dantrolene', 'cyproheptadine', 'phenylephrine', 'terbutaline', 'fentanyl', 'fluoxetine', 'moxisylyte', 'zonisamide', 'doxepin', 'sodiumvalproate', 'thioridazine', 'clomifene', 'fesoterodine', 'clonazepam', 'orciprenaline', 'fusidicacid', 'metronidazole', 'tramadol', 'ibandronicacid', 'gliclazide', 'naltrexone', 'chlortalidone', 'ciprofloxacin', 'orphenadrine', 'sulpiride', 'aminophylline', 'aspirin', 'glimepiride', 'budesonide', 'eprosartan', 'carvedilol', 'rupatadine', 'acemetacin', 'ibuprofen', 'aceclofenac', 'ketorolac', 'glyceryltrinitrate', 'olsalazine', 'zafirlukast', 'flurbiprofen', 'trimethoprim', 'cefixime', 'fosinopril', 'inositolnicotinate', 'entacapone', 'lercanidipine', 'acetazolamide', 'indapamide', 'metoprolol', 'febuxostat', 'captopril', 'disulfiram', 'propylthiouracil', 'tolbutamide', 'ezetimibe', 'risedronate', 'bisoprolol', 'pantoprazole', 'flavoxate', 'cinnarizine', 'nateglinide', 'hydrochlorothiazideandramipril', 'magnesium', 'eletriptan', 'trifluoperazine', 'chloramphenicol', 'alimemazine', 'moxonidine', 'oxcarbazepine', 'cetirizine', 'pseudoephedrine', 'albuterol', 'hydroxychloroquine', 'trimipramine', 'trimethoprimandsulfamethoxazole', 'nifedipine', 'sotalol', 'hyoscinebutylbromide', 'loratadine', 'erythromycin', 'naftidrofuryl', 'modafinil', 'esomeprazole', 'cisapride', 'oxytetracycline', 'tinidazole', 'sumatriptan', 'calcitriol', 'ranitidine', 'leflunomide', 'famciclovir', 'oxybutynin', 'perphenazine', 'ropinirole', 'ursodeoxycholicacid', 'citalopram', 'riluzole', 'darifenacin', 'buspirone', 'alendronatesodium', 'theophylline', 'paracetamolandcodeine', 'sulfasalazine', 'pre-exposureprophylaxis', 'lamotrigine', 'dexamethasone', 'stalevo', 'norfloxacin', 'ofloxacin', 'mianserin', 'nitrofurantoin', 'memantine', 'lorazepam', 'anticholinergic', 'midazolam', 'dronedarone', 'enalapril', 'fluticasone', 'lithium', 'mebendazole', 'diclofenac', 'quinapril', 'itraconazole', 'pyridostigmine', 'methylphenidate', 'trihexyphenidyl', 'co-amilofruse', 'mizolastine', 'irbesartan', 'glipizide', 'acamprosate', 'cyproterone', 'methotrexate', 'tolfenamicacid', 'prasugrel', 'mefloquine', 'methylprednisolone', 'isotretinoin', 'ibuprofenandcodeine', 'letrozole', 'nicardipine', 'magnesiumhydroxide', 'vancomycin', 'ipratropium', 'prednisone', 'potassiumchloride', 'saxagliptin', 'risperidone', 'dipipanone', 'tadalafil', 'astemizole', 'simvastatin', 'acarbose', 'quinine', 'levothyroxine', 'triamterene', 'nystatin', 'chlordiazepoxide', 'pramipexole', 'dihydrocodeine', 'allopurinol', 'terfenadine', 'mesalazine', 'hydrocortisonebuccaltablets', 'doxycycline', 'trandolapril', 'dicyclomine', 'asilone', 'nadolol', 'frovatriptan', 'ranolazine', 'azithromycin', 'cholestyramine', 'zopiclone', 'vigabatrin', 'tiotropium', 'bromocriptine', 'tiaprofenicacid', 'lidocaine', 'phenelzine', 'levofloxacin', 'diflunisal', 'hydralazine', 'acitretin', 'metoclopramideandaspirin', 'ciclosporin', 'methyldopa', 'tripotassiumdicitratobismuthate', 'galantamine', 'cyclobenzaprine', 'codeine', 'rifampicinandisoniazid', 'valaciclovir', 'phenytoin', 'co-amilozide', 'etoricoxib', 'sulindac', 'liothyronine', 'atenolol', 'cinacalcet', 'co-beneldopa', 'lansoprazole', 'indoramin', 'fenofibrate', 'zolmitriptan', 'hydrochlorothiazideandlisinopril', 'trospium', 'sucralfate', 'etamsylate', 'eplerenone', 'cromolynsodium', 'benzydamine', 'atorvastatin', 'cefalexin', 'minocycline', 'hydroxycarbamide', 'pioglitazone', 'rasagiline', 'carbamazepine', 'pivmecillinam', 'isosorbidedinitrate', 'formoterol', 'pentoxifylline', 'cabergoline', 'diazepam', 'furosemide', 'amiloride', 'tranexamicacid', 'ramipril', 'lofepramine', 'etidronicacid', 'promazine', 'aliskren', 'penicillamine', 'chlorpheniramine', 'prochlorperazine', 'bismuthsubsalicylate', 'cyclopenthiazide', 'finasteride', 'ironandfolicacid', 'penicillin', 'cyclizine', 'sertraline', 'ampicillin', 'dexketoprofen', 'nefazodone', 'solifenacin', 'felodipine', 'fluvoxamine', 'bupropion', 'tramacet', 'amoxicillin', 'tolterodine', 'isosorbidemononitrateandisosorbidedinitrate', 'betamethasone', 'cefradine', 'co-amoxiclav', 'clarithromycin', 'indometacin', 'almotriptan', 'escitalopram', 'chloroquine', 'ketotifen', 'oseltamivir', 'tamsulosin', 'olanzapine', 'levocetirizine', 'fluphenazine', 'chloralhydrate', 'tizanidine', 'cerivastatin', 'pholcodine', 'bezafibrate', 'methadone', 'bendroflumethiazide', 'methocarbamol', 'rivaroxaban', 'dosulepin', 'verapamil', 'mirabegron', 'salbutamol', 'amitriptyline', 'omeprazole', 'baclofen', 'fishoil', 'venlafaxine', 'meptazinol', 'cefpodoxime', 'fludrocortisone', 'levetiracetam', 'rosiglitazone', 'isoniazid', 'empagliflozin', 'carbimazole', 'lisinopril', 'doxazosin', 'co-magaldrox', 'temazepam', 'imipramine', 'gaviscon', 'mycophenolicacid', 'cefadroxil', 'methenamine', 'nicorandil', 'loperamide', 'mometasone', 'sterculia', 'cimetidine', 'hydroxyurea', 'celecoxib', 'meperidine', 'imidapril', 'terazosin', 'atropine', 'flecainide', 'thyroxine', 'hyoscinehydrobromide', 'dantronanddocusate', 'dabigatran', 'fluconazole', 'prucalopride', 'mebeverine', 'macrogol', 'lactulose', 'glibenclamide', 'sibutramine', 'meloxicam', 'valdecoxib', 'manevac', 'topiramate', 'ketoprofen', 'primidone', 'paroxetine', 'acrivastine', 'sodiumfusidate', 'strontium', 'zolpidem', 'oxprenolol', 'sulfinpyrazone', 'carisoprodol', 'pizotifen', 'clemastine', 'rivastigmine', 'bumetanide', 'agomelatine', 'famotidine', 'reboxetine', 'pregabalin', 'alprostadil', 'haloperidol', 'candesartan', 'kaolin', 'ivabradine', 'linaclotide', 'atenololandnifedipine', 'repaglinide', 'selegiline', 'olmesartan', 'oxazepam', 'paracetamol', 'dutasteride', 'prazosin', 'sodiumcromoglicate', 'amantadine', 'nefopam', 'mercaptopurine', 'insulin', 'docusate', 'buprenorphine', 'isosorbidemononitrate', 'chlorphenamine', 'diphenhydramine', 'valganciclovir', 'azathioprine', 'proguanil', 'propranolol', 'prednisolone', 'immunoglobin', 'metoclopramide', 'celiprolol', 'rofecoxib', 'trazodone', 'tamoxifen', 'raloxifene', 'moclobemide', 'metolazone', 'omega-3', 'dipyridamole', 'alverine', 'alendronicacid', 'quetiapine', 'duloxetine', 'simeticone', 'beclometasone', 'azapropazone', 'lymecycline', 'aripiprazole', 'domperidone', 'rabeprazole', 'hydrocortisone', 'phenobarbital', 'betahistine', 'salmeterol', 'hydroxyzine', 'loprazolam', 'prilocaine', 'triamcinolone', 'zoledronicacid', 'anastrozole', 'fybogel', 'canagliflozin', 'clopidogrel', 'balsalazide', 'vardenafil', 'lumiracoxib', 'labetalol', 'brompheniramine', 'danazol', 'clodronicacid', 'warfarin', 'methysergide', 'hydrochlorothiazide', 'sodiumpicosulfate', 'ondansetron', 'adalimumab', 'rifampicin', 'tenoxicam', 'colesevelam', 'ciprofibrate', 'chlorpromazine', 'pancrelipase', 'co-danthrusate', 'disopyramide', 'oxycodone', 'valsartan', 'nitrazepam', 'tacrolimus', 'lacosamide', 'flupentixol', 'cyclophosphamide', 'nitrousoxide', 'ethambutol', 'rosuvastatin', 'clobazam', 'donepezil', 'promethazine', 'spironolactone', 'co-careldopa', 'clindamycin', 'naproxen', 'senna', 'cefaclor', 'fluvastatin', 'propiverine', 'timolol', 'dutasterideandtamsulosin', 'dapagliflozin', 'migril', 'pilocarpine', 'aciclovir', 'morphine', 'sevelamer', 'etodolac', 'dapsone', 'sitagliptin', 'nedocromil', 'propafenone', 'oxitropium', 'cefuroxime']
print(len(codes))

537


In [4]:
timeline = 'ever taken'
model = 'COX'
lag = "0"

results = []

for ndd in ndd_list:
    
    #Load df
    df = pd.read_csv(f'{ndd}_with_tenure_OCT_09_24.csv', parse_dates = True)
    
    # Find codes to use so we don't have to use EVERYTHING
    codes_with_data = []
    lag = '0'

    for code in codes:
        m = df[['TOWNSEND', 'age_at_tenure', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        if n == 0:
            pass
        elif n_pairs < 10:
            pass
        elif n == n_pairs:
            pass
        else:
            print(code)
            codes_with_data.append(code)
    
    print(ndd)
    print(len(codes_with_data))
    
    for code in codes_with_data:
        
        m = df[['TOWNSEND', 'age_at_tenure', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        
        cph = CoxPHFitter()
        cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
        #cph.print_summary()
        #cph.plot()
        
        actual_p = cph._compute_p_values()
        results_df = cph.summary
        results_df = results_df.reset_index()
        test = results_df.iloc[3]

        covariate = code
        HR = test['exp(coef)']
        ci_min = test['exp(coef) lower 95%']
        ci_max = test['exp(coef) upper 95%']
        p = actual_p[3]

        print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
        results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

fexofenadine
losartan
metformin
colchicine
phenoxymethylpenicillin
sildenafil
mirtazapine
alfuzosin
pravastatin
amlodipine
diltiazem
indomethacin
perindopril
digoxin
bisacodyl
carbocisteine
gabapentin
flucloxacillin
fluoxetine
metronidazole
tramadol
gliclazide
ciprofloxacin
aspirin
ibuprofen
trimethoprim
indapamide
metoprolol
bisoprolol
pantoprazole
cinnarizine
cetirizine
pseudoephedrine
nifedipine
hyoscinebutylbromide
loratadine
erythromycin
esomeprazole
oxytetracycline
ranitidine
famciclovir
oxybutynin
citalopram
nitrofurantoin
memantine
lorazepam
fluticasone
diclofenac
itraconazole
risperidone
tadalafil
simvastatin
quinine
levothyroxine
nystatin
dihydrocodeine
allopurinol
doxycycline
zopiclone
galantamine
codeine
etoricoxib
atenolol
lansoprazole
atorvastatin
cefalexin
carbamazepine
diazepam
furosemide
ramipril
lofepramine
prochlorperazine
finasteride
penicillin
cyclizine
sertraline
solifenacin
felodipine
amoxicillin
tolterodine
cefradine
co-amoxiclav
clarithromycin
escitalopram
tams

KeyboardInterrupt: 

In [10]:
#Combine results
output = pd.concat([cox1])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
197,rasagiline,PD,COX,ever taken,0,645.044975,487.285191,853.879878,0.000000,62,73,0.000000,True
173,ropinirole,PD,COX,ever taken,0,340.531132,266.607129,434.952555,0.000000,87,175,0.000000,True
44,memantine,AD,COX,ever taken,0,314.076500,238.735061,413.194641,0.000000,63,77,0.000000,True
367,co-beneldopa,DEM,COX,ever taken,0,121.339002,94.663301,155.531798,0.000000,68,88,0.000000,True
473,donepezil,DEM,COX,ever taken,0,145.611781,125.977153,168.306635,0.000000,259,313,0.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,rosuvastatin,DEM,COX,ever taken,0,0.996875,0.711800,1.396121,0.985468,35,3370,0.992155,False
33,nifedipine,AD,COX,ever taken,0,0.995623,0.572696,1.730874,0.987596,13,3319,0.992155,False
77,felodipine,AD,COX,ever taken,0,1.004080,0.589030,1.711586,0.988063,14,3876,0.992155,False
32,pseudoephedrine,AD,COX,ever taken,0,0.997821,0.621882,1.601021,0.992784,18,5182,0.994835,False


In [11]:
output.to_csv(f'ALL_results_OCT_09_2024.csv', header = True, index = False)

In [12]:
! dx upload ALL_results_OCT_09_2024.csv --path data/results/ALL_results_OCT_09_2024.csv

[===========================================================>] Uploaded 69,345 of 69,345 bytes (100%) ALL_results_OCT_09_2024.csv
ID                                file-Gv3Q450JQPBbjpJ6pq1BQZf7
Class                             file
Project                           project-GfqxP70JQPBxpGq82J5XkyG1
Folder                            /data/results
Name                              ALL_results_OCT_09_2024.csv
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Wed Oct  9 22:22:12 2024
Created by                        klevine22
 via the job                      job-Gv39Zx0JQPBy6564P4f3Yfgp
Last modified                     Wed Oct  9 22:22:13 2024
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"


In [8]:
timeline = 'ever taken'
model = 'COX'
lag = "0"

results = []

for ndd in ndd_list:
    
    #Load df
    #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure_end_2018_JULY_2024.csv', parse_dates = True)
    #df = pd.read_csv(f'{ndd}_with_tenure_end_2018_drugs_1999.csv', parse_dates = True)
            
    for code in codes_with_data:
        
        m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        
        cph = CoxPHFitter()
        cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
        #cph.print_summary()
        #cph.plot()
        
        actual_p = cph._compute_p_values()
        results_df = cph.summary
        results_df = results_df.reset_index()
        test = results_df.iloc[3]

        covariate = code
        HR = test['exp(coef)']
        ci_min = test['exp(coef) lower 95%']
        ci_max = test['exp(coef) upper 95%']
        p = actual_p[3]

        print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
        results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

oxytetracycline AD 1.1973801057399136 0.830605990809804 1.7261121801251567 0.3343736354958121 31 8953
diclofenac AD 0.8571154501942013 0.7041569498818425 1.0432999277858186 0.12421573865522514 160 58280
amoxicillin AD 0.8250444956823035 0.6806726312283085 1.0000378869755813 0.050045140522821836 237 83461
propranolol AD 1.4003723485466586 0.9583531044879651 2.046263225308633 0.0818347446033603 29 9209
finasteride AD 0.9643013043319816 0.5939316448260088 1.565629670749008 0.8831229498926813 18 3460
beclometasone AD 1.0801653478681172 0.8750918987302844 1.3332967433797052 0.4728368403295484 122 36824
levothyroxine AD 0.6017047556879418 0.3999252267506045 0.9052907613734034 0.014794591307316432 25 11998
co-proxamol AD 1.417542165010503 1.0010836320573802 2.007250668411176 0.04929092216689191 35 6739
atorvastatin AD 0.8463769400024026 0.6755970997508504 1.060327116312386 0.14691032714738217 101 27564
ibuprofen AD 0.9427697614101256 0.7772864548606395 1.1434842553491578 0.5495459482702028 19

In [9]:
cox1

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N
0,oxytetracycline,AD,COX,ever taken,0,1.197380,0.830606,1.726112,3.343736e-01,31,8953
1,diclofenac,AD,COX,ever taken,0,0.857115,0.704157,1.043300,1.242157e-01,160,58280
2,amoxicillin,AD,COX,ever taken,0,0.825044,0.680673,1.000038,5.004514e-02,237,83461
3,propranolol,AD,COX,ever taken,0,1.400372,0.958353,2.046263,8.183474e-02,29,9209
4,finasteride,AD,COX,ever taken,0,0.964301,0.593932,1.565630,8.831229e-01,18,3460
...,...,...,...,...,...,...,...,...,...,...,...
167,duloxetine,AD,COX,ever taken,0,2.992373,1.754715,5.102993,5.703649e-05,14,1667
168,solifenacin,AD,COX,ever taken,0,1.288564,0.859515,1.931784,2.197522e-01,25,5062
169,galantamine,AD,COX,ever taken,0,380.733947,255.113167,568.211904,5.221378e-186,27,28
170,memantine,AD,COX,ever taken,0,232.296338,177.758392,303.567039,0.000000e+00,67,81


In [10]:
#Combine results
output = pd.concat([cox1])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
170,memantine,AD,COX,ever taken,0,232.296338,177.758392,303.567039,0.000000e+00,67,81,0.000000e+00,True
171,rivastigmine,AD,COX,ever taken,0,266.413654,183.057156,387.727180,4.515260e-187,30,37,3.883123e-185,True
169,galantamine,AD,COX,ever taken,0,380.733947,255.113167,568.211904,5.221378e-186,27,28,2.993590e-184,True
147,risperidone,AD,COX,ever taken,0,28.606363,18.769222,43.599251,7.559451e-55,23,315,3.250564e-53,True
149,mirtazapine,AD,COX,ever taken,0,6.232911,4.754679,8.170727,4.633124e-40,62,4331,1.593795e-38,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,tramadol,AD,COX,ever taken,0,0.991989,0.766555,1.283719,9.512382e-01,70,20579,9.738867e-01,False
151,fluticasonepropionateandsalmeterolxinafoate,AD,COX,ever taken,0,1.012129,0.603988,1.696069,9.634921e-01,15,3916,9.805955e-01,False
21,salbutamol,AD,COX,ever taken,0,1.004102,0.799728,1.260706,9.718753e-01,97,31288,9.828928e-01,False
165,rosuvastatin,AD,COX,ever taken,0,0.992250,0.582276,1.690882,9.771783e-01,14,3350,9.828928e-01,False


In [11]:
ndd

'AD'

In [12]:
output.to_csv(f'{ndd}_results_AUG19_2024', header = True, index = False)

In [14]:
test = output[output['SIGNIFICANT'] == True]
test

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
170,memantine,AD,COX,ever taken,0,232.296338,177.758392,303.567039,0.000000e+00,67,81,0.000000e+00,True
171,rivastigmine,AD,COX,ever taken,0,266.413654,183.057156,387.727180,4.515260e-187,30,37,3.883123e-185,True
169,galantamine,AD,COX,ever taken,0,380.733947,255.113167,568.211904,5.221378e-186,27,28,2.993590e-184,True
147,risperidone,AD,COX,ever taken,0,28.606363,18.769222,43.599251,7.559451e-55,23,315,3.250564e-53,True
149,mirtazapine,AD,COX,ever taken,0,6.232911,4.754679,8.170727,4.633124e-40,62,4331,1.593795e-38,True
102,lorazepam,AD,COX,ever taken,0,10.954256,7.457029,16.091627,3.122716e-34,28,943,8.951786e-33,True
30,sertraline,AD,COX,ever taken,0,4.554020,3.517655,5.895715,1.214246e-30,70,7221,2.983576e-29,True
98,citalopram,AD,COX,ever taken,0,3.724025,2.972787,4.665105,2.710261e-30,103,15121,5.827061e-29,True
157,quetiapine,AD,COX,ever taken,0,14.801499,8.123936,26.967761,1.321607e-18,11,420,2.525737e-17,True
136,haloperidol,AD,COX,ever taken,0,10.829654,6.089408,19.259905,5.064200e-16,12,584,8.710425e-15,True


In [8]:
codes_with_data2 = []
lag = '1-5'

for code in codes_with_data:
    df = df[~df['BIRTH_YEAR'].isna()]
    m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
    n=sum(m[f'QC{lag}_'+ code])
    df_pair = m[m[f'QC{lag}_'+ code]==1]
    n_pairs = sum(df_pair[ndd])
    if n == 0:
        pass
    elif n_pairs < 5:
        pass
    elif n == n_pairs:
        pass
    else:
        #print(code)
        codes_with_data2.append(code)

In [9]:
print(len(codes_with_data2))

125


In [10]:
timeline = '1-5 years before'
model = 'COX'
lag = "1-5"

results = []

for ndd in ndd_list:
    
    #Load df
    #df = pd.read_csv(f'{ndd}_with_tenure_end_2018_drugs_1999.csv', parse_dates = True) 
    #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure_end_2018_JULY_2024.csv', parse_dates = True)
    #df = df[~df['BIRTH_YEAR'].isna()]
    
    for code in codes_with_data2:
        
        m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        
        cph = CoxPHFitter()
        cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
        #cph.print_summary()
        #cph.plot()
        
        actual_p = cph._compute_p_values()
        results_df = cph.summary
        results_df = results_df.reset_index()
        test = results_df.iloc[3]

        covariate = code
        HR = test['exp(coef)']
        ci_min = test['exp(coef) lower 95%']
        ci_max = test['exp(coef) upper 95%']
        p = actual_p[3]

        print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
        results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox2 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

oxytetracycline PD 4.224731561025149 2.0948086482746056 8.520280254438916 5.6730785781297364e-05 8 815
diclofenac PD 2.505659626924373 1.6904167275709991 3.714072431725326 4.778734458789245e-06 27 4440
amoxicillin PD 1.1160529042267757 0.7862923037167128 1.584110742462232 0.5389086875699 35 13084
propranolol PD 6.905800661853168 4.383926705233433 10.878394185815266 7.779493689282249e-17 20 1692
finasteride PD 1.4754536185477185 0.7798852600760368 2.791389313183465 0.2318068199744619 10 1652
beclometasone PD 1.2567184161682028 0.8479758504027726 1.8624836742534105 0.2549483444057924 27 8958
levothyroxine PD 1.0655425397507132 0.47417150413231934 2.3944519949506877 0.8778655498554454 6 2448
atorvastatin PD 0.7313784549969192 0.5101379344810492 1.048568256304711 0.0887663495818358 33 14661
ibuprofen PD 1.5529353002492265 1.1405772136394168 2.11437508826348 0.005184755146150077 47 12636
lisinopril PD 0.8342740202307606 0.3447710850750938 2.018768890321526 0.6877734557083217 5 2063
glyceryl

In [11]:
cox2

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N
0,oxytetracycline,PD,COX,1-5 years before,1-5,4.224732,2.094809,8.520280,5.673079e-05,8,815
1,diclofenac,PD,COX,1-5 years before,1-5,2.505660,1.690417,3.714072,4.778734e-06,27,4440
2,amoxicillin,PD,COX,1-5 years before,1-5,1.116053,0.786292,1.584111,5.389087e-01,35,13084
3,propranolol,PD,COX,1-5 years before,1-5,6.905801,4.383927,10.878394,7.779494e-17,20,1692
4,finasteride,PD,COX,1-5 years before,1-5,1.475454,0.779885,2.791389,2.318068e-01,10,1652
...,...,...,...,...,...,...,...,...,...,...,...
120,tadalafil,PD,COX,1-5 years before,1-5,4.762795,2.855111,7.945126,2.256376e-09,16,1181
121,pregabalin,PD,COX,1-5 years before,1-5,1.780351,0.882264,3.592632,1.073396e-01,8,1839
122,duloxetine,PD,COX,1-5 years before,1-5,3.092994,1.278256,7.484111,1.226285e-02,5,723
123,solifenacin,PD,COX,1-5 years before,1-5,1.348042,0.694671,2.615939,3.772798e-01,9,2309


In [12]:
codes_with_data3 = []
lag = '5-10'

for code in codes_with_data:
    df = df[~df['BIRTH_YEAR'].isna()]
    m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
    n=sum(m[f'QC{lag}_'+ code])
    df_pair = m[m[f'QC{lag}_'+ code]==1]
    n_pairs = sum(df_pair[ndd])
    if n == 0:
        pass
    elif n_pairs < 5:
        pass
    elif n == n_pairs:
        pass
    else:
        #print(code)
        codes_with_data3.append(code)

In [13]:
print(len(codes_with_data3))

120


In [14]:
timeline = '5-10 years before'
model = 'COX'
lag = "5-10"

results = []

for ndd in ndd_list:
    
    #Load df
    #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure_end_2018_JULY_2024.csv', parse_dates = True)
    #df = pd.read_csv(f'{ndd}_with_tenure_end_2018_drugs_1999.csv', parse_dates = True)
            
    for code in codes_with_data3:
    #for code in test_list:
        
        m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        
        cph = CoxPHFitter()
        cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
        #cph.print_summary()
        #cph.plot()
        
        actual_p = cph._compute_p_values()
        results_df = cph.summary
        results_df = results_df.reset_index()
        test = results_df.iloc[3]

        covariate = code
        HR = test['exp(coef)']
        ci_min = test['exp(coef) lower 95%']
        ci_max = test['exp(coef) upper 95%']
        p = actual_p[3]

        print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
        results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox3 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

diclofenac PD 2.1899949517460175 1.6470421784899048 2.9119338601700773 6.949360694855358e-08 57 12693
amoxicillin PD 1.4508868245262019 1.105042529473456 1.904969737759121 0.007385733710724711 64 20017
propranolol PD 2.858889292074965 1.5204489194028714 5.375549207894988 0.001111900886313729 10 2103
finasteride PD 1.665499736280152 0.819419167691383 3.3851897550363175 0.15864979344240443 8 1103
beclometasone PD 0.5820612340394576 0.34083926239778833 0.9940030904542135 0.04748003232598068 14 9787
levothyroxine PD 1.48357691744659 0.7342601333766506 2.9975758861625237 0.2716811240090474 8 2597
atorvastatin PD 1.02651763252486 0.6205583504329166 1.698048941166177 0.9188212586407379 16 5146
ibuprofen PD 0.9851843841697434 0.7103859395583719 1.3662830537092354 0.928716137373305 41 16067
lisinopril PD 1.6457562657865656 0.9245158383330803 2.9296563391052803 0.09041382280174003 12 2721
glyceryltrinitrate PD 1.3078862399254751 0.7784148594875755 2.197499695358398 0.31066942962337674 15 4059
si

In [15]:
cox3

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N
0,diclofenac,PD,COX,5-10 years before,5-10,2.189995,1.647042,2.911934,6.949361e-08,57,12693
1,amoxicillin,PD,COX,5-10 years before,5-10,1.450887,1.105043,1.904970,7.385734e-03,64,20017
2,propranolol,PD,COX,5-10 years before,5-10,2.858889,1.520449,5.375549,1.111901e-03,10,2103
3,finasteride,PD,COX,5-10 years before,5-10,1.665500,0.819419,3.385190,1.586498e-01,8,1103
4,beclometasone,PD,COX,5-10 years before,5-10,0.582061,0.340839,0.994003,4.748003e-02,14,9787
...,...,...,...,...,...,...,...,...,...,...,...
115,tiotropium,PD,COX,5-10 years before,5-10,1.200415,0.494893,2.911731,6.861755e-01,5,1261
116,tadalafil,PD,COX,5-10 years before,5-10,1.252258,0.641305,2.445250,5.100019e-01,9,2271
117,rosuvastatin,PD,COX,5-10 years before,5-10,2.219305,1.049209,4.694309,3.701128e-02,7,1057
118,ezetimibe,PD,COX,5-10 years before,5-10,1.540484,0.686637,3.456108,2.946037e-01,6,1282


In [16]:
codes_with_data4 = []
lag = '10+'

for code in codes_with_data:
    df = df[~df['BIRTH_YEAR'].isna()]
    m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
    n=sum(m[f'QC{lag}_'+ code])
    df_pair = m[m[f'QC{lag}_'+ code]==1]
    n_pairs = sum(df_pair[ndd])
    if n == 0:
        pass
    elif n_pairs < 5:
        pass
    elif n == n_pairs:
        pass
    else:
        #print(code)
        codes_with_data4.append(code)

In [17]:
print(len(codes_with_data4))

116


In [18]:
timeline = '10+ years before'
model = 'COX'
lag = "10+"

results = []

for ndd in ndd_list:
    
    #Load df
    #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure_end_2018_JULY_2024.csv', parse_dates = True)
            
    for code in codes_with_data4:
        
        m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        
        cph = CoxPHFitter()
        cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
        #cph.print_summary()
        #cph.plot()
        
        actual_p = cph._compute_p_values()
        results_df = cph.summary
        results_df = results_df.reset_index()
        test = results_df.iloc[3]

        covariate = code
        HR = test['exp(coef)']
        ci_min = test['exp(coef) lower 95%']
        ci_max = test['exp(coef) upper 95%']
        p = actual_p[3]

        print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
        results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox4 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

oxytetracycline PD 0.5520153747739609 0.27373827788777866 1.11318364511579 0.09684549989203631 8 6201
diclofenac PD 0.6501747990022969 0.5003419848622981 0.8448766684531148 0.0012765410765217527 71 40949
amoxicillin PD 0.6054672502675392 0.4727970535389271 0.7753656424095127 7.005982735815202e-05 84 49592
propranolol PD 1.4764201741235108 0.878143014372419 2.4823024209977276 0.14162452676581702 15 5369
beclometasone PD 0.8509554368462438 0.6045748550254545 1.1977427600221542 0.35476401149522696 37 17642
levothyroxine PD 0.826386789208713 0.47182584953831574 1.447388111624071 0.5048541994408453 13 6859
co-proxamol PD 1.4291098451771318 0.9415949107831598 2.169037795545754 0.0934906036597634 24 6627
atorvastatin PD 0.7293907899703748 0.45259397157526976 1.1754706379360822 0.19498385003579088 18 7094
ibuprofen PD 0.7386035527249181 0.553477542883229 0.9856501227782718 0.03957627374256799 56 26830
lisinopril PD 0.7157307131941277 0.3811549165229735 1.3439953982031303 0.2981847605279141 10 

In [19]:
cox4

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N
0,oxytetracycline,PD,COX,10+ years before,10+,0.552015,0.273738,1.113184,0.096845,8,6201
1,diclofenac,PD,COX,10+ years before,10+,0.650175,0.500342,0.844877,0.001277,71,40949
2,amoxicillin,PD,COX,10+ years before,10+,0.605467,0.472797,0.775366,0.000070,84,49592
3,propranolol,PD,COX,10+ years before,10+,1.476420,0.878143,2.482302,0.141625,15,5369
4,beclometasone,PD,COX,10+ years before,10+,0.850955,0.604575,1.197743,0.354764,37,17642
...,...,...,...,...,...,...,...,...,...,...,...
111,esomeprazole,PD,COX,10+ years before,10+,1.714293,0.913436,3.217305,0.093332,10,2279
112,candesartan,PD,COX,10+ years before,10+,1.348028,0.557018,3.262335,0.507787,5,1204
113,rabeprazole,PD,COX,10+ years before,10+,1.764683,0.940081,3.312594,0.077117,10,2153
114,celecoxib,PD,COX,10+ years before,10+,1.443964,0.858626,2.428337,0.165974,15,3784


In [20]:
#Combine results
output = pd.concat([cox1, cox2, cox3, cox4])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
191,ropinirole,PD,COX,ever taken,0,291.582561,226.966008,374.595255,0.000000e+00,84,172,0.000000e+00,True
113,co-beneldopa,PD,COX,1-5 years before,1-5,461.379233,344.780031,617.410457,0.000000e+00,61,67,0.000000e+00,True
217,rasagiline,PD,COX,ever taken,0,560.498048,419.708378,748.515109,0.000000e+00,58,69,0.000000e+00,True
175,co-careldopa,PD,COX,ever taken,0,416.460794,320.605765,540.974654,0.000000e+00,80,112,0.000000e+00,True
116,ropinirole,PD,COX,1-5 years before,1-5,282.805595,210.652495,379.672714,9.392732e-309,54,101,1.091435e-306,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,hypromellose,PD,COX,ever taken,0,1.003142,0.649039,1.550437,9.887328e-01,22,7803,9.955871e-01,False
47,doxazosin,PD,COX,1-5 years before,1-5,1.005052,0.415239,2.432644,9.910853e-01,5,1562,9.959105e-01,False
25,mebeverine,PD,COX,ever taken,0,1.001728,0.687170,1.460279,9.928352e-01,30,13713,9.959105e-01,False
40,co-amoxiclav,PD,COX,10+ years before,10+,1.001437,0.680081,1.474642,9.941963e-01,28,11767,9.959105e-01,False


In [21]:
ndd

'PD'

In [22]:
output.to_csv(f'{ndd}_results_AUG_tenure_2018_drugs_1999.csv', header = True, index = False)

In [23]:
! dx upload {ndd}_results_AUG_tenure_2018_drugs_1999.csv --path hampton_files/cox_results/{ndd}_results_AUG_tenure_2018_drugs_1999.csv

[===========================================================>] Uploaded 85,392 of 85,392 bytes (100%) PD_results_AUG_tenure_2018_drugs_1999.csv
ID                                file-Gpy126jJQPBkzbxgq649p6ZV
Class                             file
Project                           project-GfqxP70JQPBxpGq82J5XkyG1
Folder                            /hampton_files/cox_results
Name                              PD_results_AUG_tenure_2018_drugs_1999.csv
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Wed Aug 14 02:06:19 2024
Created by                        klevine22
 via the job                      job-GpxqVB8JQPBQg9jXGkFFF0xq
Last modified                     Wed Aug 14 02:06:20 2024
Media type                        
archivalState                     "live"
cloudAccount        

In [32]:
ndd = 'PD'
df = pd.read_csv(f'{ndd}_results_AUG_tenure_2018_drugs_1999.csv')
df

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
0,ropinirole,PD,COX,ever taken,0,291.582561,226.966008,374.595255,0.000000e+00,84,172,0.000000e+00,True
1,co-beneldopa,PD,COX,1-5 years before,1-5,461.379233,344.780031,617.410457,0.000000e+00,61,67,0.000000e+00,True
2,rasagiline,PD,COX,ever taken,0,560.498048,419.708378,748.515109,0.000000e+00,58,69,0.000000e+00,True
3,co-careldopa,PD,COX,ever taken,0,416.460794,320.605765,540.974654,0.000000e+00,80,112,0.000000e+00,True
4,ropinirole,PD,COX,1-5 years before,1-5,282.805595,210.652495,379.672714,9.392732e-309,54,101,1.091435e-306,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,hypromellose,PD,COX,ever taken,0,1.003142,0.649039,1.550437,9.887328e-01,22,7803,9.955871e-01,False
577,doxazosin,PD,COX,1-5 years before,1-5,1.005052,0.415239,2.432644,9.910853e-01,5,1562,9.959105e-01,False
578,mebeverine,PD,COX,ever taken,0,1.001728,0.687170,1.460279,9.928352e-01,30,13713,9.959105e-01,False
579,co-amoxiclav,PD,COX,10+ years before,10+,1.001437,0.680081,1.474642,9.941963e-01,28,11767,9.959105e-01,False


In [33]:
test = df[df['LAG'] == '10+']
test = test[test['SIGNIFICANT'] == True]
print(len(test))
#test = test[test['N_pairs'] > 19]
print(len(test))
test

6
6


,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
50,amoxicillin,PD,COX,10+ years before,10+,0.605467,0.472797,0.775366,0.000070,84,49592,0.000798,True
54,simvastatin,PD,COX,10+ years before,10+,0.466853,0.317970,0.685447,0.000101,29,16531,0.001070,True
84,diclofenac,PD,COX,10+ years before,10+,0.650175,0.500342,0.844877,0.001277,71,40949,0.008726,True
96,hydrocortisone,PD,COX,10+ years before,10+,0.533186,0.348885,0.814844,0.003659,23,16497,0.021914,True
109,rofecoxib,PD,COX,10+ years before,10+,1.886856,1.185632,3.002807,0.007401,19,3676,0.039091,True
114,aspirin,PD,COX,10+ years before,10+,0.607627,0.419901,0.879281,0.008235,32,13461,0.041505,True


In [27]:
ndd = 'AD'
df = pd.read_csv(f'{ndd}_results_AUG_tenure_2018_drugs_1999.csv')
df

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
0,donepezil,AD,COX,1-5 years before,1-5,235.281545,187.772053,294.811738,0.000000e+00,113,148,0.000000e+00,True
1,memantine,AD,COX,ever taken,0,232.296338,177.758392,303.567039,0.000000e+00,67,81,0.000000e+00,True
2,memantine,AD,COX,1-5 years before,1-5,184.970600,130.322259,262.534760,1.179960e-187,35,45,2.501515e-185,True
3,rivastigmine,AD,COX,ever taken,0,266.413654,183.057156,387.727180,4.515260e-187,30,37,7.179263e-185,True
4,galantamine,AD,COX,ever taken,0,380.733947,255.113167,568.211904,5.221378e-186,27,28,6.641593e-184,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,salbutamol,AD,COX,ever taken,0,1.004102,0.799728,1.260706,9.718753e-01,97,31288,9.780264e-01,False
632,rosuvastatin,AD,COX,ever taken,0,0.992250,0.582276,1.690882,9.771783e-01,14,3350,9.818095e-01,False
633,rabeprazole,AD,COX,ever taken,0,0.994214,0.513526,1.924853,9.862649e-01,9,2535,9.889243e-01,False
634,amlodipine,AD,COX,1-5 years before,1-5,1.002904,0.699199,1.438527,9.874279e-01,32,8409,9.889243e-01,False


In [28]:
#I assume if we included the people who went on to develop AD later, donepezil would also be significant for ever taken
d = df[df['PRIOR'] == 'donepezil']
d

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
0,donepezil,AD,COX,1-5 years before,1-5,235.281545,187.772053,294.811738,0.000000e+00,113,148,0.000000e+00,True
7,donepezil,AD,COX,5-10 years before,5-10,155.504724,93.883481,257.571607,1.503548e-85,16,24,1.195321e-83,True
483,donepezil,AD,COX,ever taken,0,1.704672,0.189505,15.334188,6.341536e-01,184,238,8.329536e-01,False


In [30]:
test = df[df['LAG'] == '10+']
test = test[test['SIGNIFICANT'] == True]
print(len(test))
#test = test[test['N_pairs'] > 19]
print(len(test))
test

6
6


,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
52,diclofenac,AD,COX,10+ years before,10+,0.618204,0.488317,0.782640,0.000064,87,40962,0.000771,True
60,clomipramine,AD,COX,10+ years before,10+,5.538267,2.291677,13.384262,0.000144,5,331,0.001497,True
70,amoxicillin,AD,COX,10+ years before,10+,0.673742,0.544871,0.833093,0.000267,119,49625,0.002387,True
82,ibuprofen,AD,COX,10+ years before,10+,0.655057,0.507058,0.846254,0.001206,71,26841,0.009238,True
84,paroxetine,AD,COX,10+ years before,10+,2.118587,1.335794,3.360107,0.001421,19,3708,0.010634,True
98,fluoxetine,AD,COX,10+ years before,10+,1.748338,1.196664,2.554341,0.003876,29,7766,0.024898,True


# EXTRA CODE -- can ignore for now

In [ ]:
# #High users -- two or more times
# timeline = 'two+'
# model = 'COX'
# lag = "0"

# results = []

# for ndd in ndd_list:
    
#     #Load df
#     #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure.csv', parse_dates = True)
#     df = pd.read_csv(f'{ndd}_with_tenure_end_2018_MAY_2024.csv', parse_dates = True)
    
#     #Only selected codes with data
#     codes_with_data = []
#     for code in codes:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, code + '_two+']]
#         n=sum(m[code + '_two+'])
#         df_pair = m[m[code + '_two+']==1]
#         n_pairs = sum(df_pair[ndd])
#         if n == 0:
#             pass
#         elif n_pairs < 5:
#             pass
#         elif n == n_pairs:
#             covariate = code
#             HR = np.nan
#             ci_min = np.nan
#             ci_max = np.nan
#             p = np.nan
#             print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#             results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
#             pass
#         else:
#             codes_with_data.append(code)
            
#     for code in codes_with_data:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, code + '_two+']]
#         n=sum(m[code + '_two+'])
#         df_pair = m[m[code + '_two+']==1]
#         n_pairs = sum(df_pair[ndd])
        
#         cph = CoxPHFitter()
#         cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.1)
#         #cph.print_summary()
#         #cph.plot()
        
#         actual_p = cph._compute_p_values()
#         results_df = cph.summary
#         results_df = results_df.reset_index()
#         test = results_df.iloc[3]

#         covariate = code
#         HR = test['exp(coef)']
#         ci_min = test['exp(coef) lower 95%']
#         ci_max = test['exp(coef) upper 95%']
#         p = actual_p[3]

#         print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#         results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
# cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))


In [ ]:
# #Combine results
# output = pd.concat([cox1])

# #Adding FDR Correction

# #Sort P-values
# output = output.sort_values(by = "P_VAL")

# #Drop Nan-values
# output = output.dropna()

# #FDR Correction
# rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
# output['P_CORR'] = p_corr
# output['REJECTED'] = rejected

# output2 = output
# output2

In [ ]:
# #High users -- ten or more times
# timeline = 'ten+'
# model = 'COX'
# lag = "0"

# results = []

# for ndd in ndd_list:
    
#     #Load df
#     #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure.csv', parse_dates = True)
#     #df = pd.read_csv(f'{ndd}_with_tenure_MAY_2024.csv', parse_dates = True)
#     df = pd.read_csv(f'{ndd}_with_tenure_end_2018_MAY_2024.csv', parse_dates = True)
    
#     #Only selected codes with data
#     codes_with_data = []
#     for code in codes:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, code + '_ten+']]
#         n=sum(m[code + '_ten+'])
#         df_pair = m[m[code + '_ten+']==1]
#         n_pairs = sum(df_pair[ndd])
#         if n == 0:
#             pass
#         elif n_pairs < 5:
#             pass
#         elif n == n_pairs:
#             covariate = code
#             HR = np.nan
#             ci_min = np.nan
#             ci_max = np.nan
#             p = np.nan
#             print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#             results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
#             pass
#         else:
#             codes_with_data.append(code)
            
#     for code in codes_with_data:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, code + '_ten+']]
#         n=sum(m[code + '_ten+'])
#         df_pair = m[m[code + '_ten+']==1]
#         n_pairs = sum(df_pair[ndd])
        
#         cph = CoxPHFitter()
#         cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.1)
#         #cph.print_summary()
#         #cph.plot()
        
#         actual_p = cph._compute_p_values()
#         results_df = cph.summary
#         results_df = results_df.reset_index()
#         test = results_df.iloc[3]

#         covariate = code
#         HR = test['exp(coef)']
#         ci_min = test['exp(coef) lower 95%']
#         ci_max = test['exp(coef) upper 95%']
#         p = actual_p[3]

#         print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#         results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
# cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

In [ ]:
# #Combine results
# output = pd.concat([cox1])

# #Adding FDR Correction

# #Sort P-values
# output = output.sort_values(by = "P_VAL")

# #Drop Nan-values
# output = output.dropna()

# #FDR Correction
# rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
# output['P_CORR'] = p_corr
# output['REJECTED'] = rejected

# output3 = output
# output3

In [ ]:
# df = pd.concat([output1, output2, output3])
# df = df.sort_values(by = ['PRIOR', 'HR'])
# #df = df.sort_values(by = 'PRIOR')
# df

In [ ]:
# df.TIMELINE.value_counts()

In [ ]:
# df.to_csv('ad_prelim_results_end_2018_may_16_2024.csv', header = True, index = False)

In [ ]:
# ! dx upload ad_prelim_results_end_2018_may_16_2024.csv --path results/ad_prelim_results_end_2018_may_16_2024.csv


In [ ]:
#predictor_meaning = codes

In [ ]:
# # Now nothing left to do is run the regressions and call it a day. 
# from statsmodels.stats.multitest import fdrcorrection
# results = []

# # for predictor in range(1, 10):
# for predictor in range(len(predictor_list)):
#   predictor_name = predictor_list[predictor]
#   predictor_description = predictor_meaning[predictor]
#   this_formula = ndd + "~ df['" + predictor_list[predictor] + "']" + " + AGE_OF_RECRUIT + TOWNSEND + GENETIC_SEX"
#   fitted = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit()
#   beta_coef  = fitted.params.loc["df['" + predictor_name + "']"]
#   beta_se  = fitted.bse.loc["df['" + predictor_name + "']"]
#   p_val = fitted.pvalues.loc["df['" + predictor_name + "']"]
#   odds_ratio = np.exp(fitted.params.loc["df['" + predictor_name + "']"])
#   conf = fitted.conf_int().loc["df['" + predictor_name + "']"]
#   m5, m95 = np.exp(conf)
#   n = sum(df[predictor_name])
#   df2 = df[df[predictor_name]==1]
#   n_pairs = sum(df2[ndd])  

#   print(predictor_name, odds_ratio, m5, m95, p_val, n_pairs, n)
#   results.append((ndd, predictor_name, predictor_description, odds_ratio, m5, m95, p_val, n_pairs))

# output = pd.DataFrame(results, columns=('NDD','CODE', 'DESCRIPTION','odds_ratio', 'ci_min', "ci_max", 'P_VAL', "N_pairs"))

In [ ]:
#predictor_list = ['QC0_atorvastatin_DATE', 'QC0_pravastatin_DATE', 'QC0_rosuvastatin_DATE', 'QC0_simvastatin_DATE']
#predictor_meaning = ['atorvastatin', 'pravastatin', 'rosuvastatin', 'simvastatin']
# predictor_list = ['QC0_atorvastatin_DATE']
# predictor_meaning = ['atorvastatin']

In [ ]:
# # Now nothing left to do is run the regressions and call it a day. 
# from statsmodels.stats.multitest import fdrcorrection
# results = []

# # for predictor in range(1, 10):
# for predictor in range(len(predictor_list)):
#   predictor_name = predictor_list[predictor]
#   predictor_description = predictor_meaning[predictor]
#   this_formula = ndd + "~ df['" + predictor_list[predictor] + "']" + " + AGE_OF_RECRUIT + TOWNSEND + GENETIC_SEX + QC_atorvastatin_N"
#   fitted = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit()
#   beta_coef  = fitted.params.loc["df['" + predictor_name + "']"]
#   beta_se  = fitted.bse.loc["df['" + predictor_name + "']"]
#   p_val = fitted.pvalues.loc["df['" + predictor_name + "']"]
#   odds_ratio = np.exp(fitted.params.loc["df['" + predictor_name + "']"])
#   conf = fitted.conf_int().loc["df['" + predictor_name + "']"]
#   m5, m95 = np.exp(conf)
#   n = sum(df[predictor_name])
#   df2 = df[df[predictor_name]==1]
#   n_pairs = sum(df2[ndd])  

#   print(predictor_name, odds_ratio, m5, m95, p_val, n_pairs, n)
#   results.append((ndd, predictor_name, predictor_description, odds_ratio, m5, m95, p_val, n_pairs))

# output = pd.DataFrame(results, columns=('NDD','CODE', 'DESCRIPTION','odds_ratio', 'ci_min', "ci_max", 'P_VAL', "N_pairs"))

# Add in N of exposures

In [ ]:
# timeline = 'ever taken plus N'
# model = 'COX'
# lag = "0"

# results = []

# for ndd in ndd_list:
    
#     #Load df
#     #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure.csv', parse_dates = True)
#     df = pd.read_csv(f'{ndd}_with_tenure_lags.csv', parse_dates = True)
#     #df = pd.read_csv(f'{ndd}_with_tenure_lags_60.csv', parse_dates = True)
    
#     #Only selected codes with data
#     codes_with_data = []
#     for code in codes:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
#         if n == 0:
#             pass
#         elif n_pairs < 5:
#             pass
#         elif n == n_pairs:
#             covariate = code + '_DATE'
#             HR = np.nan
#             ci_min = np.nan
#             ci_max = np.nan
#             p = np.nan
#             print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#             results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
#             pass
#         else:
#             codes_with_data.append(code)
            
#     for code in codes_with_data:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'QC_' + code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
        
#         cph = CoxPHFitter()
#         cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.1)
#         cph.print_summary()
#         #cph.plot()
        
#         actual_p = cph._compute_p_values()
#         results_df = cph.summary
#         results_df = results_df.reset_index()
#         test = results_df.iloc[4]

#         covariate = code
#         HR = test['exp(coef)']
#         ci_min = test['exp(coef) lower 95%']
#         ci_max = test['exp(coef) upper 95%']
#         p = actual_p[4]

#         print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#         results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
# cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

In [ ]:
# #Combine results
# output = pd.concat([cox1])

# #Adding FDR Correction

# #Sort P-values
# output = output.sort_values(by = "P_VAL")

# #Drop Nan-values
# output = output.dropna()

# #FDR Correction
# rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
# output['P_CORR'] = p_corr
# output['REJECTED'] = rejected

# output2 = output
# output2

# change number of times a drug was taken
# Low prescriptions

In [ ]:
# timeline = 'take 10 or less times'
# model = 'COX'
# lag = "0"

# results = []

# for ndd in ndd_list:
    
#     #Load df
#     #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure.csv', parse_dates = True)
#     df = pd.read_csv(f'{ndd}_with_tenure_lags.csv', parse_dates = True)
#     #df = pd.read_csv(f'{ndd}_with_tenure_lags_60.csv', parse_dates = True)
    
#     #Only selected codes with data
#     codes_with_data = []
#     for code in codes:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
#         if n == 0:
#             pass
#         elif n_pairs < 5:
#             pass
#         elif n == n_pairs:
#             covariate = code + '_DATE'
#             HR = np.nan
#             ci_min = np.nan
#             ci_max = np.nan
#             p = np.nan
#             print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#             results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
#             pass
#         else:
#             codes_with_data.append(code)
            
#     for code in codes_with_data:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', code + '_N', 'QC_' + code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         m = m[m[code + '_N'] <= 10]
#         m = m[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'QC_' + code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         m = m[~m[f'QC{lag}_' + code + '_DATE'].isna()]
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
        
#         cph = CoxPHFitter()
#         cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.1)
#         cph.print_summary()
#         #cph.plot()
        
#         actual_p = cph._compute_p_values()
#         results_df = cph.summary
#         results_df = results_df.reset_index()
#         test = results_df.iloc[4]

#         covariate = code
#         HR = test['exp(coef)']
#         ci_min = test['exp(coef) lower 95%']
#         ci_max = test['exp(coef) upper 95%']
#         p = actual_p[4]

#         print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#         results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
# cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

In [ ]:
# #Combine results
# output = pd.concat([cox1])

# #Adding FDR Correction

# #Sort P-values
# output = output.sort_values(by = "P_VAL")

# #Drop Nan-values
# output = output.dropna()

# #FDR Correction
# rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
# output['P_CORR'] = p_corr
# output['REJECTED'] = rejected

# output3 = output
# output3

# No significant results if you take 10 or less times

In [ ]:
# #Try more than 10 times
# codes = ['atorvastatin', 'pravastatin','rosuvastatin', 'simvastatin']
# #codes = ['rosuvastatin']

In [ ]:
# timeline = 'take more than 10 times'
# model = 'COX'
# lag = "0"

# results = []

# for ndd in ndd_list:
    
#     #Load df
#     #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure.csv', parse_dates = True)
#     df = pd.read_csv(f'{ndd}_with_tenure_lags.csv', parse_dates = True)
#     #df = pd.read_csv(f'{ndd}_with_tenure_lags_60.csv', parse_dates = True)
    
#     #Only selected codes with data
#     codes_with_data = []
#     for code in codes:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX',  code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         #m = m[m[code + '_N'] > 10]
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
#         if n == 0:
#             pass
#         elif n_pairs < 5:
#             pass
#         elif n == n_pairs:
#             covariate = code + '_DATE'
#             HR = np.nan
#             ci_min = np.nan
#             ci_max = np.nan
#             p = np.nan
#             print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#             results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
#             pass
#         else:
#             codes_with_data.append(code)
            
#     for code in codes_with_data:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', code + '_N', 'QC_' + code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         m1 = m[m[code + '_N'] > 10]
#         m2 = m[m[code + '_N'] == 0]
#         m = pd.concat([m1, m2])
#         m = m[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'QC_' + code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         m = m[~m[f'QC{lag}_' + code + '_DATE'].isna()]
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
        
#         cph = CoxPHFitter()
#         cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.1)
#         cph.print_summary()
#         #cph.plot()
        
#         actual_p = cph._compute_p_values()
#         results_df = cph.summary
#         results_df = results_df.reset_index()
#         test = results_df.iloc[4]

#         covariate = code
#         HR = test['exp(coef)']
#         ci_min = test['exp(coef) lower 95%']
#         ci_max = test['exp(coef) upper 95%']
#         p = actual_p[4]

#         print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#         results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
# cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

In [ ]:
# #Combine results
# output = pd.concat([cox1])

# #Adding FDR Correction

# #Sort P-values
# output = output.sort_values(by = "P_VAL")

# #Drop Nan-values
# output = output.dropna()

# #FDR Correction
# rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
# output['P_CORR'] = p_corr
# output['REJECTED'] = rejected

# output4 = output
# output4

# More than 10 times without number

In [ ]:
# timeline = 'take more than 10 times'
# model = 'COX'
# lag = "0"

# results = []

# for ndd in ndd_list:
    
#     #Load df
#     #df = pd.read_csv(f'files_for_cox/{ndd}_with_tenure.csv', parse_dates = True)
#     df = pd.read_csv(f'{ndd}_with_tenure_lags.csv', parse_dates = True)
#     #df = pd.read_csv(f'{ndd}_with_tenure_lags_60.csv', parse_dates = True)
    
#     #Only selected codes with data
#     codes_with_data = []
#     for code in codes:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX',  code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         m1 = m[m[code + '_N'] > 10]
#         m2 = m[m[code + '_N'] == 0]
#         m = pd.concat([m1, m2])
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
#         if n == 0:
#             pass
#         elif n_pairs < 5:
#             pass
#         elif n == n_pairs:
#             covariate = code + '_DATE'
#             HR = np.nan
#             ci_min = np.nan
#             ci_max = np.nan
#             p = np.nan
#             print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#             results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
#             pass
#         else:
#             codes_with_data.append(code)
            
#     for code in codes_with_data:
#         m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', code + '_N', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         m = m[m[code + '_N'] > 10]
#         #m = df[['TOWNSEND', 'AGE_OF_RECRUIT', 'GENETIC_SEX', 'tenure', ndd, f'QC{lag}_' + code + '_DATE']]
#         #m = m[m[code + '_N'] <= 100]
#         m = m[~m[f'QC{lag}_' + code + '_DATE'].isna()]
#         n=sum(m[f'QC{lag}_'+ code + '_DATE'])
#         df_pair = m[m[f'QC{lag}_'+ code + '_DATE']==1]
#         n_pairs = sum(df_pair[ndd])
        
#         cph = CoxPHFitter()
#         cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.1)
#         cph.print_summary()
#         #cph.plot()
        
#         actual_p = cph._compute_p_values()
#         results_df = cph.summary
#         results_df = results_df.reset_index()
#         test = results_df.iloc[3]

#         covariate = code
#         HR = test['exp(coef)']
#         ci_min = test['exp(coef) lower 95%']
#         ci_max = test['exp(coef) upper 95%']
#         p = actual_p[3]

#         print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
#         results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
# cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))